# ICESat-2 Meets Google Earth Engine Map

If you run this in a new computational environment for the first time 
(including on Binder, Colab, etc.) you need to authenticate google earth engine. 
Open a terminal and run
~~~
earthengine autheniticate
~~~
Then follow the instructions. 

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
import numpy as np
import ee
import geemap
import datetime

from oautils import dataCollector

ee.Initialize()

### Specify the parameters for the ICESat-2 data
Browse OpenAltimetry to find a track / region of interest! Here's a few examples.

In [2]:
##################################################################
# # amery icesat-2 doline by grounding line
# kwargs = {'latlims': [-71.3364, -71.280],
#           'lonlims': [71.450, 71.536],
#           'track': 721,
#           'beam': 'gt3r'}
# date = '2019-11-13'

##################################################################
# # weird amery 'lake' / blue ice? 
# kwargs = {'latlims': [-72.44, -72.418],
#           'lonlims': [67.40, 67.44],
#           'track': 81,
#           'beam': 'gt3r'} # also in gt3l (strong beam)
# date = '2019-01-02'

##################################################################
# lake on Nivlisen ice shelf
# http://openaltimetry.org/data/api/icesat2/atl03?date=2020-01-16&minx=12.107692195781404&miny=-70.34956862465471&maxx=12.426364789894341&maxy=-70.2449105354736&trackId=312&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json
kwargs = {'latlims': [-70.34957, -70.24491],
          'lonlims': [12.10769, 12.42636],
          'track': 312,
          'beam': 'gt2r'} # also in gt2l (weak beam)
date = '2020-01-16'

### Now download the data

In [3]:
is2data = dataCollector(date=date,**kwargs)
is2data.requestData()

OpenAltimetry API URL: https://openaltimetry.org/data/api/icesat2/atlXX?date=2020-01-16&minx=12.10769&miny=-70.34957&maxx=12.42636&maxy=-70.24491&trackId=312&beamName=gt2r&outputFormat=json&client=jupyter
Date: 2020-01-16
Track: 312
Beam: gt2r
Latitude limits: [-70.34957, -70.24491]
Longitude limits: [12.10769, 12.42636]
---> requesting ATL03 data... Done.
---> requesting ATL06 data... Done.
---> requesting ATL08 data... Done.


### The ATL03 / ATL06 / ATL08 data are available as pandas dataframes

In [4]:
is2data.atl06

,lat,lon,h
0,-70.245025,12.299825,45.887184
1,-70.245202,12.299762,45.831590
2,-70.245379,12.299698,45.780262
3,-70.245556,12.299635,45.789750
4,-70.245734,12.299571,45.816616
...,...,...,...
551,-70.348776,12.261042,45.261272
552,-70.348953,12.260975,45.195520
553,-70.349130,12.260908,45.111626
554,-70.349307,12.260842,45.032890


### Plot the ATL03 and ATL06 data

In [5]:
fig = plt.figure(figsize=[9,4.5],dpi=100)
ax = fig.add_subplot(1,1,1)
ax.scatter(is2data.atl03.lat,is2data.atl03.h,s=10,c='k',alpha=0.1,edgecolors='none')
h2, = ax.plot(is2data.atl06.lat,is2data.atl06.h,c='r',lw=5, alpha=0.3, label='ATL06 land ice elevation')
h1 = ax.scatter(-9999,-9999,s=20,c='k',alpha=0.5,edgecolors='none',label='ATL03 photon cloud')
# h2, = ax.plot([-9999, -9998],[-9999, -9998],c='r',lw=1.3, label='ATL06 land ice elevation')
ax.set_xlim(kwargs['latlims'])
hrange = is2data.atl06.h.max() - is2data.atl06.h.min()
ax.set_ylim(is2data.atl06.h.min() - 0.5*hrange ,is2data.atl06.h.max() + 0.5*hrange)
ax.set_xlabel('latitude',fontsize=8)
ax.set_ylabel('elevation [m]',fontsize=8)
ax.tick_params(axis='both', labelsize=7)
ax.legend(handles=[h1,h2], loc='lower right', fontsize=7)
ax.set_title('ICESat-2 track %d %s on %s' % (kwargs['track'],kwargs['beam'].upper(),date),fontsize=9)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'ICESat-2 track 312 GT2R on 2020-01-16')

### How long is this ground track?

In [6]:
lat1, lat2 = is2data.atl06.lat[0], is2data.atl06.lat.iloc[-1]
lon1, lon2 = is2data.atl06.lon[0], is2data.atl06.lon.iloc[-1]

def dist_latlon2meters(lat1, lon1, lat2, lon2):
    # returns the distance between two lat/lon coordinate points along the earth's surface in meters
    R = 6371000
    def deg2rad(deg):
        return deg * (np.pi/180)
    dlat = deg2rad(lat2-lat1)
    dlon = deg2rad(lon2-lon1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(deg2rad(lat1)) * np.cos(deg2rad(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

dist_xatc = dist_latlon2meters(lat1, lon1, lat2, lon2)
dist_xatc

11707.194486037968

### Add this to the map
We can use google earth satellite imagery as the basemap.

In [7]:
gtx_geom = ee.Geometry.LineString(coords=list(zip(is2data.atl06.lon,is2data.atl06.lat)), proj='EPSG:4326', geodesic=True)
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('HYBRID')
Map.addLayer(ee.FeatureCollection(gtx_geom),{'color': 'red'},'ground track')
center_lat = (lat1 + lat2) / 2
center_lon = (lon1 + lon2) / 2
Map.setCenter(center_lon, center_lat, 7);
Map

Map(center=[-70.29725446674048, 12.280300997860806], controls=(WidgetControl(options=['position', 'transparent…

### Let's look for Sentinel-2 and Landsat 8 images!
These live in image collections on Google Earth Engine. 

In [8]:
collection_name1 = 'COPERNICUS/S2_SR'  # Sentinel-2 earth engine collection 
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

collection_name2 = 'LANDSAT/LC08/C01/T2'  # Landsat 8 earth engine collection 
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2
# Note: Landsat 8 ingestion into Earth Engine seems to not have reached Antarctica yet, so using raw scenes...

# Specify for how many days around the ICESat-2 overpass we want to look 
days_buffer_imagery = 15

### Query for images that overlap spatially with - and were acquired close in time to - the ICESat-2 overpass.

In [9]:
date_requested = date
dateformat = '%Y-%m-%d'
datetime_requested = datetime.datetime.strptime(date_requested, dateformat)
search_start = (datetime_requested - datetime.timedelta(days=days_buffer_imagery)).strftime(dateformat)
search_end = (datetime_requested + datetime.timedelta(days=days_buffer_imagery)).strftime(dateformat)

# the point of interest (center of the track) as an Earth Engine Geometry
poi = ee.Geometry.Point(center_lon, center_lat)

# the region of interest around it
buffer_around_center_meters = dist_xatc*0.8
roi = poi.buffer(buffer_around_center_meters)

# the collection to query: 
# 1) merge Landsat 8 and Sentinel-2 collections
# 2) filter by acquisition date
# 3) filter by the point of interest
# 4) sort by acquisition date
collection = ee.ImageCollection(collection_name1) \
    .merge(ee.ImageCollection(collection_name2)) \
    .filterDate(search_start, search_end) \
    .filterBounds(poi) \
    .sort('system:time_start') 

# clip to the region of interest
def clip(img): return img.clip(roi)
collection = collection.map(clip);

# get info 
n_imgs = collection.size().getInfo()
info = collection.getInfo()
print('number of images +/- %d days of date requested: %d' % (days_buffer_imagery, n_imgs))

number of images +/- 15 days of date requested: 16


### Add all the images that were found to the Map as individual layers
If the image is Landsat 8, this pan-sharpens the RGB image to give better spatial resolution. 

In [10]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('HYBRID')

# get a list of images to access them in sequence (there may be a better way to do this...)
list_of_images = collection.toList(n_imgs)
for i in range(n_imgs):
    
    # get the relevant info
    thisDate = datetime.datetime.fromtimestamp(info['features'][i]['properties']['system:time_start']/1e3)
    dtstr = thisDate.strftime(dateformat)
    dt = (thisDate - datetime_requested).days
    ID = info['features'][i]['id']
    rel = 'before' if dt<0 else 'after'
    print('%02d: %s (%3d days %s ICESat-2): %s' % (i, dtstr, np.abs(dt), rel, ID))
    
    # normalize the image to [0.0, 1.0] range, based on max and min in the rgb bands
    img = ee.Image(list_of_images.get(i))
    rgb = img.select('B4', 'B3', 'B2')
    rgbmax = rgb.reduce(ee.Reducer.max()).reduceRegion(reducer=ee.Reducer.max(), geometry=rgb.geometry(), bestEffort=True, maxPixels=1e6)
    rgbmin = rgb.reduce(ee.Reducer.min()).reduceRegion(reducer=ee.Reducer.min(), geometry=rgb.geometry(), bestEffort=True, maxPixels=1e6)
    rgb = rgb.unitScale(ee.Number(rgbmin.get('min')), ee.Number(rgbmax.get('max'))).clamp(0.0, 1.0)
    
    # only show the first layer, then can toggle others on in map
    show_layer = True if i==0 else False
    
    # if the image is Landsat 8, then pan-sharpen the image
    if 'LANDSAT' in ID: 
        pan = img.select('B8').unitScale(ee.Number(rgbmin.get('min')), ee.Number(rgbmax.get('max'))).clamp(0.0, 1.0)
        huesat = rgb.rgbToHsv().select('hue', 'saturation')
        upres = ee.Image.cat(huesat, pan).hsvToRgb()
        Map.addLayer(upres, {'min': 0, 'max': 1}, name='%02d: %s (L8)'%(i,dtstr), shown=show_layer)
    
    # if the image is Sentinel-2, then just plot the
    else:
        Map.addLayer(rgb, {'min': 0, 'max': 1}, name='%02d: %s (S2)'%(i,dtstr), shown=show_layer)

# show the ground track and center the map on it
Map.addLayer(ee.FeatureCollection(gtx_geom),{'color': 'red'},'ground track')
Map.centerObject(gtx_geom)

00: 2020-01-04 ( 12 days before ICESat-2): LANDSAT/LC08/C01/T2/LC08_166109_20200105
01: 2020-01-04 ( 12 days before ICESat-2): LANDSAT/LC08/C01/T2/LC08_166110_20200105
02: 2020-01-06 ( 10 days before ICESat-2): COPERNICUS/S2_SR/20200106T080919_20200106T080922_T32DPH
03: 2020-01-06 ( 10 days before ICESat-2): COPERNICUS/S2_SR/20200106T080919_20200106T080922_T32DPG
04: 2020-01-06 ( 10 days before ICESat-2): LANDSAT/LC08/C01/T2/LC08_164110_20200107
05: 2020-01-11 (  5 days before ICESat-2): LANDSAT/LC08/C01/T2/LC08_167109_20200112
06: 2020-01-13 (  3 days before ICESat-2): LANDSAT/LC08/C01/T2/LC08_165110_20200114
07: 2020-01-16 (  0 days after ICESat-2): COPERNICUS/S2_SR/20200116T080919_20200116T080921_T32DPH
08: 2020-01-16 (  0 days after ICESat-2): COPERNICUS/S2_SR/20200116T080919_20200116T080921_T32DPG
09: 2020-01-20 (  4 days after ICESat-2): LANDSAT/LC08/C01/T2/LC08_166109_20200121
10: 2020-01-20 (  4 days after ICESat-2): LANDSAT/LC08/C01/T2/LC08_166110_20200121
11: 2020-01-22 (  6 

## Finally, display the imagery and the ground track on the map. 
Click on the Layer button on the top right to toggle the visibility of the images.

In [11]:
Map

Map(center=[-70.29725446674048, 12.280300997860806], controls=(WidgetControl(options=['position', 'transparent…